Preparing Data

<p> We will be combining data from three files </p>
<ol>
<li> We will select name, artist, genre, and number of grammy awards from grammySongs_1999-2019.csv. This information is not immediately available from the sheet, so we will group by artist and name and count the number of grammys each song has won. </li>
<li> We will select artist, name, peak position, average position, and weeks on chart from billboardHot100_1999-2019. </li>
<li> We will select artist, name, energy, danceability, tempo, and valence from data/songAtrributes_1999-2019.csv. </li>
</ol>

In [66]:
import pandas as pd
import numpy as np


In [81]:
grammyAwards = pd.read_csv("data/grammySongs_1999-2019.csv")
grammyAwards = grammyAwards[['Name', 'Artist', 'GrammyAward']]
# grammyAwards = grammyAwards[['Name', 'Artist', 'GrammyAward']]
grammyAwards = grammyAwards.groupby(['Name', 'Artist']).count().reset_index()
# namesOnly = grammyAwards.groupby(['Name', 'Artist'])
# grammyAwards = namesOnly['Genre'].agg(lambda x: "".join(x)).reset_index(name="Genre")
grammyAwards.head()

,Name,Artist,GrammyAward
0,'Til Summer Comes Around,Keith Urban,1
1,24 Frames,"Ason Isbell, Songwriter",1
2,24k Magic,Bruno Mars,1
3,500 Miles High,"Chick Corea, Soloist",1
4,99 Problems,Jay-z,1


In [24]:
billboardHot100 = pd.read_csv('data/billboardHot100_1999-2019.csv')
billboardHot100 = billboardHot100[['Artists', 'Name', 'Peak.position', 'Weeks.on.chart','Weekly.rank']]
billboardHot100['Weeks.on.chart'] = billboardHot100['Weeks.on.chart'].fillna(1)
billboardHot100['Peak.position'] = billboardHot100['Peak.position'].fillna(100)
billboardHot100['Weekly.rank'] = billboardHot100['Weekly.rank'].fillna(100)
# billboardHot100.groupby(by=['Artist', 'Name']).min('Peak.position').max('Weeks.on.chart')
billboardHot100 = billboardHot100.groupby(['Artists', 'Name']).agg({'Peak.position':'min', 'Weeks.on.chart':'max', 'Weekly.rank':'mean'})[['Peak.position','Weeks.on.chart','Weekly.rank']].reset_index()
billboardHot100.rename(columns = {'Weekly.rank':'Avg.rank'}, inplace = True)
billboardHot100

,Artists,Name,Peak.position,Weeks.on.chart,Avg.rank
0,"""Weird Al"" Yankovic",Canadian Idiot,82.0,3.0,91.333333
1,"""Weird Al"" Yankovic",Word Crimes,39.0,2.0,65.000000
2,'N Sync,Bye Bye Bye,4.0,23.0,14.260870
3,'N Sync,Gone,11.0,24.0,27.125000
4,'N Sync,It's Gonna Be Me,1.0,25.0,21.680000
...,...,...,...,...,...
7208,will.i.am,Fall Down,100.0,1.0,58.000000
7209,will.i.am,I Got It From My Mama,31.0,9.0,56.666667
7210,will.i.am,It's A New Day,78.0,2.0,81.000000
7211,"will.i.am, Britney Spears",Scream & Shout,3.0,24.0,14.583333


In [7]:
songAtrributes = pd.read_csv('data/songAttributes_1999-2019.csv')
songAtrributes = songAtrributes[['Name', 'Energy', 'Danceability', 'Tempo', 'Valence']]

In [9]:
songAtrributes.head()

,Name,Energy,Danceability,Tempo,Valence
0,Welcome All Again,0.904,0.520,106.022,0.365
1,Fuzzy,0.709,0.581,120.027,0.408
2,Dig,0.918,0.572,144.061,0.370
3,You,0.661,0.596,111.975,0.183
4,My Days,0.808,0.520,92.721,0.666


In [13]:
combined_df = grammyAwards.merge(songAtrributes, on='Name', how = 'left')
combined_df.head(100)

,Name,Artist,GrammyAward,Energy,Danceability,Tempo,Valence
0,'Til Summer Comes Around,Keith Urban,1,0.629,0.570,127.907,0.308
1,24 Frames,"Ason Isbell, Songwriter",1,NaN,NaN,NaN,NaN
2,24k Magic,Bruno Mars,1,NaN,NaN,NaN,NaN
3,500 Miles High,"Chick Corea, Soloist",1,NaN,NaN,NaN,NaN
4,99 Problems,Jay-z,1,0.544,0.939,98.919,0.568
...,...,...,...,...,...,...,...
95,Angel,Lalah Hathaway,1,0.720,0.621,113.975,0.273
96,Angel,Lalah Hathaway,1,0.586,0.572,135.979,0.373
97,Angel,Lalah Hathaway,1,0.586,0.572,135.979,0.373
98,Angel,Lalah Hathaway,1,0.589,0.567,135.966,0.358
